<a href="https://colab.research.google.com/github/vksepm/hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/blob/main/code_04_XX%20A%20simple%20workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#The code below is required when running as an executable
#This is NOT needed in a notebook

#async def main():
#    <async code>

#if __name__ == "__main__":
#    import asyncio
#    asyncio.run(main())

In [ ]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    Event,
    Context,
    step,
)
from llama_index.utils.workflow import draw_all_possible_flows
from typing import Any

#Define a custom ValidateEvent that inherits from Event
class ValidateEvent(Event):
    iterations: int

#Define a custom ContinueEvent that inherits from Event
class ContinueEvent(Event):
    iterations: int

#Define a workflow class inheriting from Workflow
class SimpleWorkFlow(Workflow):

    #Any initialization steps needed here
    def __init__ (
        self,
        *args: Any,
        max_iterations: int, #Pass custom parameters too.
        **kwargs: Any,
    ) -> None:

        #Initialize the super class
        super().__init__(*args, **kwargs)
        #Store input into instance variables
        self.max_iterations = max_iterations


    #First step
    @step
    async def runLoop(self,
                       ctx: Context,
                       event : StartEvent | ContinueEvent ) -> ValidateEvent :

        #If StartEvent, the initialize variables
        if isinstance(event, StartEvent):
            iterations=0
            current_result=""
        else:
            #for ContinueEvent
            #read current result from Context
            current_result=await ctx.get("current_result")
            #Read current iteration count from event
            iterations=event.iterations

        #Bump no. of iterations
        iterations = iterations + 1
        #Create current result value
        current_result = f"*** Iteration : { iterations } {self.max_iterations}"
        print(current_result)

        #Set current result value in context.
        await ctx.set("current_result", current_result)

        #Return validate event, with current value of iterations
        return ValidateEvent(iterations=iterations)

    @step
    async def checkIterations(self,
                              ctx: Context,
                              event: ValidateEvent) -> StopEvent | ContinueEvent :

        #Read current iteration count from event
        iterations = event.iterations
        #Read max iterations from instance variable
        max_iterations=self.max_iterations
        #Read current result from context
        current_result=await ctx.get("current_result")

        print(f"*** Current iteration to validate : {iterations} {max_iterations}")
        #Perform check if max iterations is reached.
        if iterations >= max_iterations:
            #Return stop event if max iterations is reached, with current result
            return StopEvent(result=current_result)
        else:
            #Read continue event with current iteration count
            return ContinueEvent(iterations=iterations)

#Draw a workflow graph. This may not work inside codespaces.
#draw_all_possible_flows(SimpleWorkFlow, filename="test_flow.html")


In [ ]:
#Create a workflow
workflowobj = SimpleWorkFlow(
                timeout = 10, # Max allowed time to execute workflow, in seconds
                verbose = True, # Print debug messages
                max_iterations=3 # Custom input to workflow
                )

#Execute the workflow with async
result = await workflowobj.run()
print(result)

Running step runLoop
*** Iteration : 1 3
Step runLoop produced event ValidateEvent
Running step checkIterations
*** Current iteration to validate : 1 3
Step checkIterations produced event ContinueEvent
Running step runLoop
*** Iteration : 2 3
Step runLoop produced event ValidateEvent
Running step checkIterations
*** Current iteration to validate : 2 3
Step checkIterations produced event ContinueEvent
Running step runLoop
*** Iteration : 3 3
Step runLoop produced event ValidateEvent
Running step checkIterations
*** Current iteration to validate : 3 3
Step checkIterations produced event StopEvent
*** Iteration : 3 3
